# Test Suite: Schema Creation and Architecture

**Purpose:** Validate the three-tier medallion architecture schema creation

**Scope:**
- Schema existence (bronze, silver, gold)
- Ownership and authorization
- Privilege verification
- Architecture compliance
- Idempotency testing
- No unexpected schemas

**Testing Strategy:**
- Existence validation (all 3 schemas present)
- Naming validation (exact case-sensitive matches)
- Ownership validation (correct user assigned)
- Privilege validation (usage and creation rights)
- Architecture validation (3-tier medallion pattern)
- Isolation validation (no extra schemas)

**Prerequisites:**
- PostgreSQL server running
- sql_retail_analytics_warehouse database exists
- `setup/create_schemas.sql` has been executed
- Connection credentials available
- Required packages: psycopg2, pytest, ipytest, pandas

## Setup: Import Dependencies & Configure Connection

In [ ]:
import os
import psycopg2
from psycopg2 import sql
import pytest
import ipytest
import pandas as pd

# Configure ipytest for notebook usage
ipytest.autoconfig()

# Database connection parameters
DB_CONFIG = {
    'host': 'localhost',
    'database': 'sql_retail_analytics_warehouse',
    'user': 'postgres',
    'password': os.getenv('POSTGRES_PASSWORD', 'your_password_here')
}

# Expected schemas (medallion architecture)
EXPECTED_SCHEMAS = ['bronze', 'silver', 'gold']

print("✅ Dependencies imported successfully")

## Fixtures: Database Connections

In [ ]:
@pytest.fixture(scope='module')
def db_connection():
    """Connection to sql_retail_analytics_warehouse database."""
    conn = psycopg2.connect(**DB_CONFIG)
    conn.autocommit = True
    yield conn
    conn.close()

@pytest.fixture(scope='module')
def db_cursor(db_connection):
    """Cursor for warehouse database."""
    cursor = db_connection.cursor()
    yield cursor
    cursor.close()

print("✅ Fixtures defined")

## Test Suite 1: Schema Existence

In [ ]:
%%ipytest

def test_bronze_schema_exists(db_cursor):
    """Verify bronze schema exists."""
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name = 'bronze'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Bronze schema must exist"

def test_silver_schema_exists(db_cursor):
    """Verify silver schema exists."""
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name = 'silver'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Silver schema must exist"

def test_gold_schema_exists(db_cursor):
    """Verify gold schema exists."""
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name = 'gold'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Gold schema must exist"

def test_all_three_schemas_exist(db_cursor):
    """Verify all three medallion schemas exist together."""
    db_cursor.execute("""
        SELECT schema_name
        FROM information_schema.schemata
        WHERE schema_name IN ('bronze', 'silver', 'gold')
        ORDER BY schema_name
    """)
    
    schemas = [row[0] for row in db_cursor.fetchall()]
    assert schemas == EXPECTED_SCHEMAS, \
        f"Expected {EXPECTED_SCHEMAS}, got {schemas}"

## Test Suite 2: Schema Naming Validation

In [ ]:
%%ipytest

def test_schema_names_lowercase(db_cursor):
    """Verify schema names are lowercase (PostgreSQL standard)."""
    db_cursor.execute("""
        SELECT schema_name
        FROM information_schema.schemata
        WHERE schema_name IN ('bronze', 'silver', 'gold')
    """)
    
    schemas = [row[0] for row in db_cursor.fetchall()]
    
    for schema in schemas:
        assert schema.islower(), f"Schema '{schema}' should be lowercase"

def test_schema_names_no_special_chars(db_cursor):
    """Verify schema names contain only alphanumeric characters."""
    db_cursor.execute("""
        SELECT schema_name
        FROM information_schema.schemata
        WHERE schema_name IN ('bronze', 'silver', 'gold')
    """)
    
    schemas = [row[0] for row in db_cursor.fetchall()]
    
    for schema in schemas:
        assert schema.isalpha(), \
            f"Schema '{schema}' should contain only letters"

def test_schema_names_exact_match(db_cursor):
    """Verify schema names match exactly (case-sensitive check)."""
    db_cursor.execute("""
        SELECT nspname
        FROM pg_catalog.pg_namespace
        WHERE nspname IN ('bronze', 'silver', 'gold')
        ORDER BY nspname
    """)
    
    schemas = [row[0] for row in db_cursor.fetchall()]
    assert schemas == EXPECTED_SCHEMAS, \
        f"Schema names don't match exactly: {schemas}"

## Test Suite 3: Schema Ownership

In [ ]:
%%ipytest

def test_bronze_schema_has_owner(db_cursor):
    """Verify bronze schema has an owner assigned."""
    db_cursor.execute("""
        SELECT pg_catalog.pg_get_userbyid(nspowner) AS owner
        FROM pg_catalog.pg_namespace
        WHERE nspname = 'bronze'
    """)
    
    owner = db_cursor.fetchone()[0]
    assert owner is not None, "Bronze schema must have an owner"
    assert len(owner) > 0, "Owner name should not be empty"

def test_silver_schema_has_owner(db_cursor):
    """Verify silver schema has an owner assigned."""
    db_cursor.execute("""
        SELECT pg_catalog.pg_get_userbyid(nspowner) AS owner
        FROM pg_catalog.pg_namespace
        WHERE nspname = 'silver'
    """)
    
    owner = db_cursor.fetchone()[0]
    assert owner is not None, "Silver schema must have an owner"
    assert len(owner) > 0, "Owner name should not be empty"

def test_gold_schema_has_owner(db_cursor):
    """Verify gold schema has an owner assigned."""
    db_cursor.execute("""
        SELECT pg_catalog.pg_get_userbyid(nspowner) AS owner
        FROM pg_catalog.pg_namespace
        WHERE nspname = 'gold'
    """)
    
    owner = db_cursor.fetchone()[0]
    assert owner is not None, "Gold schema must have an owner"
    assert len(owner) > 0, "Owner name should not be empty"

def test_all_schemas_same_owner(db_cursor):
    """Verify all medallion schemas have the same owner (consistency)."""
    db_cursor.execute("""
        SELECT
            nspname,
            pg_catalog.pg_get_userbyid(nspowner) AS owner
        FROM pg_catalog.pg_namespace
        WHERE nspname IN ('bronze', 'silver', 'gold')
        ORDER BY nspname
    """)
    
    results = db_cursor.fetchall()
    owners = [row[1] for row in results]
    
    # All owners should be the same
    assert len(set(owners)) == 1, \
        f"All schemas should have same owner, got: {dict(results)}"

## Test Suite 4: Schema Privileges

In [ ]:
%%ipytest

def test_can_use_bronze_schema(db_cursor):
    """Verify current user has USAGE privilege on bronze schema."""
    # Try to set search_path (requires USAGE)
    db_cursor.execute("SET search_path TO bronze, public")
    
    db_cursor.execute("SHOW search_path")
    search_path = db_cursor.fetchone()[0]
    
    assert 'bronze' in search_path, "Should have USAGE on bronze schema"

def test_can_create_table_in_bronze(db_cursor):
    """Verify current user can create tables in bronze schema."""
    db_cursor.execute("""
        CREATE TABLE IF NOT EXISTS bronze.test_privilege_table (
            id SERIAL PRIMARY KEY,
            test_column TEXT
        )
    """)
    
    # Verify table was created
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = 'bronze'
        AND table_name = 'test_privilege_table'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Should be able to create tables in bronze schema"
    
    # Clean up
    db_cursor.execute("DROP TABLE IF EXISTS bronze.test_privilege_table")

def test_can_create_table_in_silver(db_cursor):
    """Verify current user can create tables in silver schema."""
    db_cursor.execute("""
        CREATE TABLE IF NOT EXISTS silver.test_privilege_table (
            id SERIAL PRIMARY KEY
        )
    """)
    
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = 'silver'
        AND table_name = 'test_privilege_table'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Should be able to create tables in silver schema"
    
    db_cursor.execute("DROP TABLE IF EXISTS silver.test_privilege_table")

def test_can_create_table_in_gold(db_cursor):
    """Verify current user can create tables in gold schema."""
    db_cursor.execute("""
        CREATE TABLE IF NOT EXISTS gold.test_privilege_table (
            id SERIAL PRIMARY KEY
        )
    """)
    
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.tables
        WHERE table_schema = 'gold'
        AND table_name = 'test_privilege_table'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Should be able to create tables in gold schema"
    
    db_cursor.execute("DROP TABLE IF EXISTS gold.test_privilege_table")

## Test Suite 5: Architecture Compliance

In [ ]:
%%ipytest

def test_three_tier_architecture_complete(db_cursor):
    """Verify complete 3-tier medallion architecture (bronze→silver→gold)."""
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name IN ('bronze', 'silver', 'gold')
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 3, "Must have exactly 3 medallion schemas"

def test_schema_order_logical(db_cursor):
    """Verify schemas are in logical order: bronze < gold < silver (alphabetically)."""
    db_cursor.execute("""
        SELECT schema_name
        FROM information_schema.schemata
        WHERE schema_name IN ('bronze', 'silver', 'gold')
        ORDER BY schema_name
    """)
    
    schemas = [row[0] for row in db_cursor.fetchall()]
    
    # When sorted alphabetically: bronze, gold, silver
    assert schemas == ['bronze', 'gold', 'silver'], \
        f"Alphabetical order should be bronze, gold, silver: got {schemas}"

def test_no_missing_tier(db_cursor):
    """Verify no tier is missing from the medallion architecture."""
    for schema in EXPECTED_SCHEMAS:
        db_cursor.execute("""
            SELECT COUNT(*)
            FROM information_schema.schemata
            WHERE schema_name = %s
        """, (schema,))
        
        count = db_cursor.fetchone()[0]
        assert count == 1, f"Tier '{schema}' is missing from architecture"

## Test Suite 6: Isolation and Clean State

In [ ]:
%%ipytest

def test_no_unexpected_user_schemas(db_cursor):
    """Verify no unexpected user schemas exist beyond medallion + setup + public."""
    db_cursor.execute("""
        SELECT schema_name
        FROM information_schema.schemata
        WHERE schema_name NOT IN ('pg_catalog', 'information_schema', 'pg_toast')
        ORDER BY schema_name
    """)
    
    schemas = [row[0] for row in db_cursor.fetchall()]
    
    # Expected: bronze, gold, public, setup (if seed scripts run), silver
    allowed_schemas = {'bronze', 'silver', 'gold', 'public', 'setup'}
    unexpected = set(schemas) - allowed_schemas
    
    if unexpected:
        print(f"⚠️  Unexpected schemas found: {unexpected}")
        print(f"   All user schemas: {schemas}")
    
    # This is informational - may have other schemas in development
    assert len(schemas) > 0, "Should have at least some user schemas"

def test_medallion_schemas_empty_or_structured(db_cursor):
    """Verify medallion schemas are either empty or have expected structure."""
    for schema in EXPECTED_SCHEMAS:
        db_cursor.execute("""
            SELECT COUNT(*)
            FROM information_schema.tables
            WHERE table_schema = %s
        """, (schema,))
        
        table_count = db_cursor.fetchone()[0]
        
        # Bronze may have tables (crm_*, erp_*), others may be empty
        # Just verify count is non-negative
        assert table_count >= 0, f"Table count should be >= 0 for {schema}"

## Test Suite 7: Idempotency Validation

In [ ]:
%%ipytest

def test_create_schema_if_not_exists_bronze(db_cursor):
    """Verify CREATE SCHEMA IF NOT EXISTS is idempotent for bronze."""
    # Try to create again (should succeed without error)
    db_cursor.execute("""
        CREATE SCHEMA IF NOT EXISTS bronze AUTHORIZATION CURRENT_USER
    """)
    
    # Verify still exists
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name = 'bronze'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Bronze schema should still exist after IF NOT EXISTS"

def test_create_schema_if_not_exists_silver(db_cursor):
    """Verify CREATE SCHEMA IF NOT EXISTS is idempotent for silver."""
    db_cursor.execute("""
        CREATE SCHEMA IF NOT EXISTS silver AUTHORIZATION CURRENT_USER
    """)
    
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name = 'silver'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Silver schema should still exist after IF NOT EXISTS"

def test_create_schema_if_not_exists_gold(db_cursor):
    """Verify CREATE SCHEMA IF NOT EXISTS is idempotent for gold."""
    db_cursor.execute("""
        CREATE SCHEMA IF NOT EXISTS gold AUTHORIZATION CURRENT_USER
    """)
    
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name = 'gold'
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 1, "Gold schema should still exist after IF NOT EXISTS"

def test_multiple_executions_idempotent(db_cursor):
    """Verify running create_schemas.sql multiple times is safe."""
    # Execute all three CREATE SCHEMA statements multiple times
    for _ in range(3):
        db_cursor.execute("""
            CREATE SCHEMA IF NOT EXISTS bronze AUTHORIZATION CURRENT_USER;
            CREATE SCHEMA IF NOT EXISTS silver AUTHORIZATION CURRENT_USER;
            CREATE SCHEMA IF NOT EXISTS gold AUTHORIZATION CURRENT_USER;
        """)
    
    # Should still have exactly 3 schemas
    db_cursor.execute("""
        SELECT COUNT(*)
        FROM information_schema.schemata
        WHERE schema_name IN ('bronze', 'silver', 'gold')
    """)
    
    count = db_cursor.fetchone()[0]
    assert count == 3, "Should still have exactly 3 schemas after multiple executions"

## Summary: Run All Tests

In [ ]:
# Run all tests in this notebook
ipytest.run('-v')

## Manual Inspection: Schema Details

In [ ]:
# Connect to warehouse database
conn = psycopg2.connect(**DB_CONFIG)

# Get comprehensive schema information
df_schemas = pd.read_sql("""
    SELECT
        current_database()                     AS database,
        n.nspname                              AS schema_name,
        pg_catalog.pg_get_userbyid(n.nspowner) AS owner,
        pg_catalog.obj_description(n.oid, 'pg_namespace') AS description
    FROM pg_catalog.pg_namespace AS n
    WHERE n.nspname IN ('bronze', 'silver', 'gold')
    ORDER BY n.nspname
""", conn)

print("\n📁 Medallion Architecture Schemas:")
display(df_schemas)

# Get table counts per schema
df_table_counts = pd.read_sql("""
    SELECT
        table_schema AS schema_name,
        COUNT(*) AS table_count,
        STRING_AGG(table_name, ', ' ORDER BY table_name) AS tables
    FROM information_schema.tables
    WHERE table_schema IN ('bronze', 'silver', 'gold')
    AND table_type = 'BASE TABLE'
    GROUP BY table_schema
    ORDER BY table_schema
""", conn)

print("\n📊 Tables per Schema:")
if len(df_table_counts) > 0:
    display(df_table_counts)
else:
    print("   No tables found in medallion schemas (clean state)")

# Get all user schemas
df_all_schemas = pd.read_sql("""
    SELECT
        n.nspname                              AS schema_name,
        pg_catalog.pg_get_userbyid(n.nspowner) AS owner,
        CASE
            WHEN n.nspname IN ('bronze', 'silver', 'gold') THEN 'Medallion Architecture'
            WHEN n.nspname = 'public' THEN 'Default Schema'
            WHEN n.nspname = 'setup' THEN 'Setup Procedures'
            ELSE 'Other'
        END AS purpose
    FROM pg_catalog.pg_namespace AS n
    WHERE n.nspname NOT IN ('pg_catalog', 'information_schema', 'pg_toast')
    ORDER BY
        CASE
            WHEN n.nspname IN ('bronze', 'silver', 'gold') THEN 1
            WHEN n.nspname = 'public' THEN 2
            WHEN n.nspname = 'setup' THEN 3
            ELSE 4
        END,
        n.nspname
""", conn)

print("\n🗂️  All User Schemas:")
display(df_all_schemas)

# Get schema privileges for current user
df_privileges = pd.read_sql("""
    SELECT
        n.nspname AS schema_name,
        HAS_SCHEMA_PRIVILEGE(CURRENT_USER, n.nspname, 'USAGE') AS can_use,
        HAS_SCHEMA_PRIVILEGE(CURRENT_USER, n.nspname, 'CREATE') AS can_create
    FROM pg_catalog.pg_namespace AS n
    WHERE n.nspname IN ('bronze', 'silver', 'gold')
    ORDER BY n.nspname
""", conn)

print("\n🔐 Current User Privileges:")
display(df_privileges)

conn.close()
print("\n✅ Inspection complete")

## Architecture Visualization

In [ ]:
# Visual representation of the medallion architecture
print("""
╔════════════════════════════════════════════════════════════╗
║           THREE-TIER MEDALLION ARCHITECTURE                ║
╠════════════════════════════════════════════════════════════╣
║                                                            ║
║  ┌──────────────────────────────────────────────────┐    ║
║  │  BRONZE LAYER (Raw Data)                         │    ║
║  │  - Source system replicas (CRM, ERP)             │    ║
║  │  - No transformations                             │    ║
║  │  - Exact CSV mappings                             │    ║
║  └──────────────────────────────────────────────────┘    ║
║                          ↓                                ║
║  ┌──────────────────────────────────────────────────┐    ║
║  │  SILVER LAYER (Cleansed & Conformed)             │    ║
║  │  - Data quality rules applied                     │    ║
║  │  - Type conversions & validation                  │    ║
║  │  - Deduplication                                  │    ║
║  └──────────────────────────────────────────────────┘    ║
║                          ↓                                ║
║  ┌──────────────────────────────────────────────────┐    ║
║  │  GOLD LAYER (Analytics Ready)                     │    ║
║  │  - Dimensional models                             │    ║
║  │  - Pre-aggregated metrics                         │    ║
║  │  - Denormalized for BI tools                      │    ║
║  └──────────────────────────────────────────────────┘    ║
║                                                            ║
╚════════════════════════════════════════════════════════════╝
""")